In [1]:
print("hello")

hello


In [90]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from collections import defaultdict
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
import pickle

import sys
import os

# Add the current directory to the Python path
current_dir = os.path.dirname(os.path.abspath("/Users/wangzeyu/Desktop/AI/part_b/agent"))
sys.path.append(current_dir)

from agent import Agent
from referee.game import \
    PlayerColor, Action, SpawnAction, SpreadAction, HexPos, HexDir, constants, Board

In [ ]:
def get_feature_vector(state: Board):
    feature_dict = defaultdict(list)
    feature_dict["num_red_cells"] = len(state._player_cells(PlayerColor.RED))
    feature_dict["num_blue_cells"] = len(state._player_cells(PlayerColor.BLUE))
    feature_dict["num_blank_cells"] = constants.MAX_TOTAL_POWER - feature_dict["num_red_cells"] - feature_dict["num_blue_cells"]
    feature_dict["red_total_power"] = state._color_power(PlayerColor.RED)
    feature_dict["blue_total_power"] = state._color_power(PlayerColor.BLUE)
    feature_dict["turn_count"] = state.turn_count
    feature_dict["turn_color"] = state.turn_color
    return feature_dict

In [29]:
df = pd.read_csv("/Users/wangzeyu/Desktop/AI/part_b/agent/game_states.csv")
# replace the unwanted index column
df.drop(columns=["Unnamed: 0"], inplace=True, errors="ignore")
df.info()
print(df.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   num_red_cells     102 non-null    int64
 1   num_blue_cells    102 non-null    int64
 2   num_blank_cells   102 non-null    int64
 3   red_total_power   102 non-null    int64
 4   blue_total_power  102 non-null    int64
 5   turn_count        102 non-null    int64
 6   turn_color        102 non-null    int64
 7   result            102 non-null    int64
dtypes: int64(8)
memory usage: 6.5 KB
   num_red_cells  num_blue_cells  num_blank_cells  red_total_power  \
0             11              18               20               18   
1             11              15               23               21   
2             17               8               24               35   
3             12              11               26               29   
4             13              14               22         

In [95]:
X = df.drop(columns=["result"], inplace=False)
y = df["result"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
clf = MLPClassifier(hidden_layer_sizes=(14,), solver='lbfgs', random_state=1).fit(X_train, y_train)
clf.predict_proba(X_test[:1])[0]
# clf.predict_proba(np.reshape([2, 1, 46, 2, 1, 3, 0], (1, -1)))
clf.score(X_test, y_test)
# clf.coefs_
# clf.intercepts
clf.predict_proba(np.reshape([2, 1, 46, 2, 1, 3, 0], (1, -1)))


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


array([[0.27290189, 0.30783253, 0.41926558]])

In [91]:
with open('clf.pkl', 'wb') as f:
    pickle.dump(clf, f)